# 自动写诗      
首先导入必要的库：

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import sys

# 加载数据集

本次实验的数据来自chinese-poetry：https://github.com/chinese-poetry/chinese-poetry

实验提供预处理过的数据集，含有57580首唐诗，每首诗限定在125词，不足125词的以```<s>```填充。数据集以npz文件形式保存，包含三个部分：
- （1）data: 诗词数据，将诗词中的字转化为其在字典中的序号表示。
- （2）ix2word: 序号到字的映射
- （3）word2ix: 字到序号的映射

预处理数据集的下载：[点击下载](https://yun.sfo2.digitaloceanspaces.com/pytorch_book/pytorch_book/tang.npz)

In [5]:
def prepareData():
    
    # 读入预处理的数据
    datas = np.load("tang.npz",allow_pickle=True)
    data = datas['data']
    ix2word = datas['ix2word'].item()
    word2ix = datas['word2ix'].item()
    
    # 转为torch.Tensor
    data = torch.from_numpy(data)
    dataloader = DataLoader(data,
                         batch_size = 16,
                         shuffle = True,
                         num_workers = 2
                         )
    
    return dataloader, ix2word, word2ix

In [6]:
dataloader, ix2word, word2ix = prepareData()

# 构建模型

模型包括Embedding层、LSTM层和输出层。

In [8]:
class PoetryModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(PoetryModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim, num_layers=2)
        self.linear = nn.Linear(self.hidden_dim, vocab_size)

    def forward(self, input, hidden = None):
        seq_len, batch_size = input.size()
        
        if hidden is None:
            h_0 = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
            c_0 = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
        else:
            h_0, c_0 = hidden

        embeds = self.embedding(input)
        output, hidden = self.lstm(embeds, (h_0, c_0))
        output = self.linear(output.view(seq_len * batch_size, -1))
        return output, hidden

# 训练模型

In [9]:
# 设置超参数
learning_rate = 5e-3       # 学习率
embedding_dim = 128        # 嵌入层维度
hidden_dim = 256           # 隐藏层维度
model_path = None          # 预训练模型路径
epochs = 4                 # 训练轮数
verbose = True             # 打印训练过程
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [72]:
def train(dataloader, ix2word, word2ix):
    # 配置模型，是否继续上一次的训练
    model = PoetryModel(len(word2ix), embedding_dim, hidden_dim)
    if model_path:
        model.load_state_dict(torch.load(model_path))
    model.to(device)
    
    # 设置优化器
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

    # 设置损失函数
    criterion = nn.CrossEntropyLoss()

    # 定义训练过程
    for epoch in range(epochs):
        # 创建tqdm对象
        pbar = tqdm(enumerate(dataloader), total=len(dataloader), file=sys.stdout)
        for batch_idx, data in pbar:
            data = data.long().transpose(1, 0).contiguous()
            data = data.to(device)
            input, target = data[:-1, :], data[1:, :]
            output, _ = model(input)
            loss = criterion(output, target.view(-1))
            
            # 更新进度条
            pbar.set_description(f'Train Epoch: {epoch+1} [{batch_idx * len(data[1])}/{len(dataloader.dataset)} ({100. * batch_idx / len(dataloader):.1f}%)] Loss: {loss.item():.6f}')
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # 关闭进度条
        pbar.close()

    # 保存模型
    torch.save(model.state_dict(), 'model1.pth')

In [73]:
train(dataloader, ix2word, word2ix)

Train Epoch: 1 [43176/57580 (100.0%)] Loss: 2.171277: 100%|██████████| 3599/3599 [01:04<00:00, 55.88it/s]
Train Epoch: 2 [43176/57580 (100.0%)] Loss: 2.025051: 100%|██████████| 3599/3599 [01:03<00:00, 56.50it/s]
Train Epoch: 3 [43176/57580 (100.0%)] Loss: 2.248627: 100%|██████████| 3599/3599 [01:03<00:00, 56.84it/s]
Train Epoch: 4 [43176/57580 (100.0%)] Loss: 1.795037: 100%|██████████| 3599/3599 [01:02<00:00, 57.24it/s]


# 生成唐诗

给定几个词，根据这几个词接着生成一首完整的唐诗。

In [75]:
def generate(start_words, ix2word, word2ix):

    # 读取模型
    model = PoetryModel(len(word2ix), embedding_dim, hidden_dim)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    
    # 读取唐诗的第一句
    results = list(start_words)
    start_word_len = len(start_words)
    
    # 设置第一个词为<START>
    input = torch.Tensor([word2ix['<START>']]).view(1, 1).long()
    input = input.to(device)
    hidden = None

    # 生成唐诗
    for i in range(max_gen_len):
        output, hidden = model(input, hidden)
        # 读取第一句
        if i < start_word_len:
            w = results[i]
            input = input.data.new([word2ix[w]]).view(1, 1)
        # 生成后面的句子
        else:
            top_index = output.data[0].topk(1)[1][0].item()
            w = ix2word[top_index]
            results.append(w)
            input = input.data.new([top_index]).view(1, 1)
        # 结束标志
        if w == '<EOP>':
            del results[-1]
            break
            
    return results

In [89]:
# 设置超参数
model_path = 'model1.pth'        # 模型路径
start_words = '湖光秋月两相和'  # 唐诗的第一句
max_gen_len = 64              # 生成唐诗的最长长度
results = generate(start_words, ix2word, word2ix)
print(results)

['湖', '光', '秋', '月', '两', '相', '和', '，', '一', '片', '风', '流', '不', '可', '寻', '。', '一', '片', '风', '流', '千', '里', '外', '，', '一', '年', '春', '色', '满', '江', '头', '。', '风', '流', '不', '见', '青', '山', '外', '，', '人', '世', '不', '知', '何', '处', '寻', '。', '山', '水', '悠', '悠', '云', '雨', '夜', '，', '月', '明', '寒', '食', '白', '云', '端', '。']


In [91]:
for i in results:
    print(i, end="")
    if i =="。":
        print('')

湖光秋月两相和，一片风流不可寻。
一片风流千里外，一年春色满江头。
风流不见青山外，人世不知何处寻。
山水悠悠云雨夜，月明寒食白云端。


# 生成藏头诗



In [3]:
def gen_acrostic(start_words, ix2word, word2ix):

    # 读取模型
    model = PoetryModel(len(word2ix), embedding_dim, hidden_dim)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    
    # 读取唐诗的“头”
    results = []
    start_word_len = len(start_words)
    
    # 设置第一个词为<START>
    input = (torch.Tensor([word2ix['<START>']]).view(1, 1).long())
    input = input.to(device)
    hidden = None

    index = 0            # 指示已生成了多少句
    pre_word = '<START>' # 上一个词

    # 生成藏头诗
    for i in range(max_gen_len_acrostic):
        output, hidden = model(input, hidden)
        top_index = output.data[0].topk(1)[1][0].item()
        w = ix2word[top_index]

        # 如果遇到标志一句的结尾，喂入下一个“头”
        if (pre_word in {u'。', u'！', '<START>'}):
            # 如果生成的诗已经包含全部“头”，则结束
            if index == start_word_len:
                break
            # 把“头”作为输入喂入模型
            else:
                w = start_words[index]
                index += 1
                input = (input.data.new([word2ix[w]])).view(1, 1)
                
        # 否则，把上一次预测作为下一个词输入
        else:
            input = (input.data.new([word2ix[w]])).view(1, 1)
        results.append(w)
        pre_word = w
        
    return results

In [10]:
# 设置超参数
model_path = 'model1.pth'                 # 模型路径
start_words_acrostic = '湖光秋月两相和'  # 唐诗的“头”
max_gen_len_acrostic = 128              # 生成唐诗的最长长度
results_acrostic = gen_acrostic(start_words_acrostic, ix2word, word2ix)
print(results_acrostic)

['湖', '水', '有', '清', '浅', '，', '山', '川', '无', '所', '依', '。', '光', '辉', '照', '云', '雨', '，', '白', '日', '照', '江', '湖', '。', '秋', '风', '吹', '白', '日', '，', '白', '日', '照', '江', '湖', '。', '月', '下', '天', '地', '间', '，', '山', '中', '有', '余', '姿', '。', '两', '岸', '不', '可', '见', '，', '高', '楼', '无', '所', '依', '。', '相', '思', '不', '可', '见', '，', '不', '见', '人', '间', '时', '。', '和', '风', '吹', '白', '日', '，', '白', '日', '照', '江', '湖', '。']


In [98]:
for i in results_acrostic:
    print(i, end="")
    if i =="。":
        print('')

湖水有清浅，山川无所依。
光辉照云雨，白日照江湖。
秋风吹白日，白日照江湖。
月下天地间，山中有余姿。
两岸不可见，高楼无所依。
相思不可见，不见人间时。
和风吹白日，白日照江湖。
